In [278]:
"""Mkt APIs"""
from fredapi import Fred
import numpy as np
import pandas as pd
import yfinance as yf
import yahooquery as yq
from fredapi import Fred
import datetime as dt
from scipy import optimize
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.trading.client import TradingClient

KEY = "PKQ9XJDJLTX686HE3ZL9"
SECRET = "4ES985YvYlOWz0eAtzdhcETJ1asEPBSw3gq9ZXs7"
ENDP = "https://paper-api.alpaca.markets"
API = TradingClient(KEY, SECRET)
STOCK_LIST = ["AAPL", "TSLA", "AMZN", "GOOG", "BA", "NKE", "AMGN", "KO", "PG", "MSFT", "CRM", "JPM"]
FRED = Fred(api_key="f3fea224d98377beff02b72fbe0cb196")
RRF = (FRED.get_series_latest_release("GS10")/12).iloc[-1]

In [279]:
"""ML libs"""
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_probability as tfp

In [280]:
def get_data(stocks, start_date, end_date):
    stock_data = yf.download(stocks, start_date, end_date)
    stock_data = pd.DataFrame(stock_data)
    closing_prices = stock_data["Close"]
    returns = closing_prices.pct_change()[1:]
    log_returns = np.log(returns+1)[1:]
    #print(log_returns)
    #mean_of_returns = log_returns.mean()
    #cov_of_returns = log_returns.cov()


    return {"returns":returns, "log_returns":log_returns}# , mean_of_returns, cov_of_returns

In [281]:
"""Data retrieval test"""
end_date = dt.datetime.now()
start_date = end_date - dt.timedelta(days=400)

data = get_data(["BA"], start_date, end_date)
log_returns_pd = data["log_returns"]
returns_pd = data["returns"]
log_returns = np.array(log_returns_pd)#log_returns_pd.iloc[1]
returns = np.array(returns_pd)
#print(log_returns)
#print(np.array(log_returns_pd))

[*********************100%%**********************]  1 of 1 completed


In [282]:
"""Build regressor"""
def build_regressor(data, i=0, ma=3, ar=7):
    regressor = tfp.sts.AutoregressiveIntegratedMovingAverage(observed_time_series=data, ar_order=ar, ma_order=ma, integration_degree=i)
    variational_posts = tfp.sts.build_factored_surrogate_posterior(model=regressor)
    # Allow external control of optimization to reduce test runtimes.
    num_variational_steps = 200 # @param { isTemplate: true}
    num_variational_steps = int(num_variational_steps)

    # Build and optimize the variational loss function.
    elbo_loss_curve = tfp.vi.fit_surrogate_posterior(
        target_log_prob_fn=regressor.joint_distribution(
            observed_time_series=data).log_prob,
        surrogate_posterior=variational_posts,
        optimizer=tf.optimizers.Adam(learning_rate=0.1),
        num_steps=num_variational_steps,
        jit_compile=True)

    return {"regressor":regressor, "variational_posteriors":variational_posts}


In [283]:
"""Sample run"""
log_model = build_regressor(log_returns)
raw_model = build_regressor(returns)
log_regressor = log_model["regressor"]
raw_regressor = raw_model["regressor"]
log_post = log_model["variational_posteriors"]
raw_post = raw_model["variational_posteriors"]



In [284]:
#print(log_regressor.ar_coefficients)
#print(q_samples_fuck_(log_regressor.parameters[0].name))
#print(log_regressor.parameters[0])
#print(log_regressor.get_parameter('ar_coefficients'))

def get_coeffs(n_samps):
    samples = log_post.sample(n_samps)
    coeffs = samples['ar_coefficients']
    coeff_means = np.mean(coeffs, axis=0)

    return {"coeffs":coeffs, "coeff_means":coeff_means}
    #print(samples)

In [285]:
"""TIME metric"""
mean_coeffs = get_coeffs(50)['coeff_means']
TIME = abs(np.sum(mean_coeffs)/(1-np.sum(mean_coeffs)))
MIM_no_norm = np.sum(np.abs(mean_coeffs))/(1+(np.sum(np.abs(mean_coeffs))))
print(mean_coeffs)
print("TIME: ", TIME)
print("MIM (no norm): ", MIM_no_norm)

[ 0.64189032 -0.51298645 -0.15916194  0.0631229   0.02492813 -0.01669281
  0.09172631]
TIME:  0.15317171352477008
MIM (no norm):  0.601674378445515
